In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [2]:
!pip install transformers sentence-transformers scikit-learn

In [3]:
from sentence_transformers import SentenceTransformer
import json
import re
import numpy as np
import os

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def preprocess_text(text):
    chunks = re.split(r"(?=\d{5})", text.strip())
    clean_chunks = [re.sub(r'\n+', ' ', chunk.strip()) for chunk in chunks if chunk.strip()]
    return clean_chunks

def load_and_preprocess(filepath):
    with open(filepath, 'r') as file:
        text = file.read()
    chunks = preprocess_text(text)
    return chunks

def convert_to_vectors(chunks):
    vectors = model.encode(chunks)
    return vectors

def save_vectors_to_json(chunks, vectors, output_json):
    data = [{'chunk': chunk, 'vector': vector.tolist()} for chunk, vector in zip(chunks, vectors)]
    with open(output_json, 'w') as json_file:
        json.dump(data, json_file)

chunks = load_and_preprocess('/content/Procore_Subcontrator Invoice.txt')
vectors = convert_to_vectors(chunks)

save_vectors_to_json(chunks, vectors, 'college_data_vectors.json')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

def load_vectors_from_json(input_json):
    with open(input_json, 'r') as json_file:
        data = json.load(json_file)
    chunks = [item['chunk'] for item in data]
    vectors = [np.array(item['vector']) for item in data]
    return chunks, np.array(vectors)

# Find the most similar chunk
def find_similar_chunk(query, chunks, vectors, top_n=5):
    # Convert the query to a vector
    query_vector = model.encode([query])[0]

    # Calculate cosine similarity between the query vector and all text vectors
    similarities = cosine_similarity([query_vector], vectors)[0]

    # Get the indices of the top N most similar chunks
    top_indices = similarities.argsort()[-top_n:][::-1]

    # Return the most similar chunks
    similar_chunks = [(chunks[i], similarities[i]) for i in top_indices]
    return similar_chunks

chunks, vectors = load_vectors_from_json('college_data_vectors.json')

In [5]:
os.environ["OPENAI_API_KEY"] = "your api key"


In [9]:
query =  input("")
similar_chunks = find_similar_chunk(query, chunks, vectors)

print(f"Query: {query}\n")

prompt = str(similar_chunks)+"using this data answer this query"+query
print(prompt)


what is the current score of match
Query: what is the current score of match

[('0', 0.24743242412809577), ('68346.55,       "completed_work_retainage_percent": 10,       "completed_work_retainage_amount": "1201.0",       "contract_sum_to_date":', 0.23516603442153095), ('79159.15,       "current_payment_due":', 0.23237016558195905), ('10812.6,       "formatted_period": "01/06/19 - 30/06/19",       "less_previous_certificates_for_payment": 0,       "negative_change_order_item_total": 0,       "negative_new_change_order_item_total": 0,       "negative_previous_change_order_item_total": 0,       "net_change_by_change_orders":', 0.20601132465413327), ('8', 0.19497300670916368)]using this data answer this querywhat is the current score of match
